# Histo2D

The **Histo2D** class provides the same features as **ROOT**'s **TH2D** class but with extended functionality and improved ease of use. So make sure to also consult the [ROOT documentation](https://root.cern.ch/doc/master/classTH2D.html).

Again we start by creating a test sample that we can play with:

In [ ]:
import ROOT
from mephisto import IOManager, Histo2D

IOManager.CreateTestSample("testsample.root", tree="TestTree", nbranches=10, nevents=1e4)

Now let's create a Histo1D, fill it with some dummy data and draw it to a canvas. You can use equidistant bins or define your own intervalls, you decide!

In [ ]:
## equidistant bins:
h = Histo2D("h", "MyScatterPlot", 50, 0.0, 10.0, 50, 0.0, 10.0)

## variable bins:
# xbinlowedges = [0, 1, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 8, 10]
# ybinlowedges = [0, 1, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 8, 10]
# h = Histo2D("h", "MyHisto", xbinlowedges, ybinlowedges)

h.Fill("testsample.root", tree="TestTree", varexp="branch_6:branch_7", cuts=["branch_6<8"], weight="branch_10")

c = ROOT.TCanvas()
c.Draw()
h.Draw("COL")


Now let's use the ```Print``` function of Histo2D instead.

In [ ]:
try:
    del c  # delete it so that Jupyter doesn't get confused...
except NameError:
    pass

h.Print("myscatterplot.png")

# view in in Jupyter:
from IPython.display import Image
Image(filename='myscatterplot.png')

That looks at bit better already. A color scale would be useful however. Well that's easy:

In [ ]:
h.Print("myscatterplot.png", drawoption="COLZ")

Image(filename='myscatterplot.png')

It seems the labels of the z-axis (color scale) have not made it into the figure. We can increase the margin to the right of the pad with the ```rightmargin``` property. This is a property of the **Pad** class (since its parent **TPad** has a method called ```SetRightMargin```) but you can also access it via the Histo2D ```Print``` function:

In [ ]:
h.Print("myscatterplot.png", drawoption="COLZ", rightmargin=0.14)

Image(filename='myscatterplot.png')

Lastly let's imagine we'd want to plot some measurement result as a function of two continuous parameters. The problem however is that we only have a few data points available, e.g.:

In [ ]:
nbinsx, nbinsy = 100, 100
xsparsity, ysparsity = 4, 4  # only one in four bins is filled

norm = float(nbinsx ** 2 + nbinsy ** 2)

g = Histo2D("grid", "MyGrid", nbinsx, 0, nbinsx, nbinsy, 0, nbinsy)
for x in range(nbinsx + 1):
    for y in range(nbinsy + 1):
        if x % xsparsity == 0 and y % ysparsity == 0:
            g.Fill(x, y, ((x + 1) ** 2 + (y + 1) ** 2) / norm)

g.Print("mygrid.png", xtitle="x", ytitle="y", rightmargin=0.14)

Image(filename='mygrid.png')

#     h1.Interpolate()  # because TH2::Smooth sucks

#     h1.Print(
#         "test_histo2d_1.png",
#         xtitle="X",
#         ytitle="Y",
#         xunits="k#AA",
#         yunits="#mub^{-2}",
#         rightmargin=0.15,
#         contour=[0.2, 0.5, 0.6],
#     )

We'd like to at least get an estimate for the values in between, i.e. all the white space. You could use TH2D's ```Smooth``` method...

In [ ]:
g.Smooth()
g.Print("mygrid.png", xtitle="x", ytitle="y", rightmargin=0.14)

Image(filename='mygrid.png')

...but yeah, it's probably not really what you were looking for. Instead use Histo2D's ```Interpolate``` method, which uses [scipy](https://www.scipy.org/) to get the job done:

In [ ]:
# reset it initial conditions:
g.Reset()
for x in range(nbinsx + 1):
    for y in range(nbinsy + 1):
        if x % xsparsity == 0 and y % ysparsity == 0:
            g.Fill(x, y, ((x + 1) ** 2 + (y + 1) ** 2) / norm)

g.Interpolate()
g.Print("mygrid.png", xtitle="x", ytitle="y", rightmargin=0.14)

Image(filename='mygrid.png')

That's much nicer, right!? Lastly, for adding countour lines for example at 0.2, 0.5 and 0.6 there's a method called ```SetContour``` and thus a property called ```contour```.

In [ ]:
g.Print("mygrid.png", xtitle="x", ytitle="y", rightmargin=0.14, contour=[0.2, 0.5, 0.6])

Image(filename='mygrid.png')

If you'd like to change the attributes of the contour lines you can add any Histo2D property with the prefix ```contour``` as a keyword argument.

Alright, that's it. Thanks for taking the tutorial!